In [78]:
from officelib.xllib import *
from officelib.const import *
from hello.hello import *
import os
os.chdir("C:/.replcache")

In [79]:
def download(ip, name, fp):
    app = open_hello(ip)
    app.login()
    r = app.getdatareport_bybatchname(name)
    with open(fp, 'wb') as f:
        f.write(r)
#download("71.189.82.196:6", "rtdtest", "rtdtest.csv")

In [80]:
def find_column(rng, header):
    return rng.Find(header, SearchOrder=xlByColumns)

In [81]:
def fudge_column(cells):
    """ For some reason, cells.Find() can't find 
    the requested header if the header occurs in the 
    first column, so insert a dummy here """
    cells.Columns(1).Insert()

In [82]:
def get_header_data(cells, h):
    c = find_column(cells, h)
    s = c.Offset(2,2)
    e = s.End(xlDown)
    rows = e.Row-s.Row + 1
    return c, s, e, rows

def make_col(ref, offset, header):
    c = ref.Offset(1, offset)
    c.Value = header
    s = c.Offset(2, 1)
    return c, s

def create_data(cells):
    rng=cells.Rows(1)
    fo = get_data("FilterOvenPV(C)")
    ta = get_data("TempA(C)")
    tb = get_data("TempB(C)")
    
    amb = tb.Offset(1,4)
    diff = amb.Offset(1,2)
    
    amb.Value = "Ambient"
    diff.Value = "Diff A-B"
    
    return fo, ta, tb, amb, diff

def make_amb_diff(cells):
    fo, fo_start, fo_end, fo_r = get_header_data(cells, "FilterOvenPV(C)")
    tempa, tempa_start, tempa_end, tempa_r = get_header_data(cells, "TempA(C)")
    tempb, tempb_start, tempb_end, tempb_r = get_header_data(cells, "TempB(C)")

    assert (fo_r == tempa_r and fo_r == tempb_r)
    
    amb, amb_start = make_col(tempb, 4, "Ambient")
    diff, diff_start = make_col(amb, 2, "Diff A-B")

    amb_form = "=%s" % addr(fo_start)
    diff_form = "=%s-%s" % (addr(tempa_start), addr(tempb_start))

    amb_start.Value = amb_form
    diff_start.Value = diff_form
    
    form_rng = cells.Range(amb_start, diff_start)
    dst = cells.Range(amb_start, diff_start.Offset(fo_r, 1))
    form_rng.AutoFill(dst)
    
    return amb, diff
    
def addr(r):
    return r.GetAddress(False, False)



In [83]:
def chart_data_rng(cells, xc, yc):
    x = cells.Range(xc, xc.End(xlDown))
    y = cells.Range(yc, yc.End(xlDown))
    return x, y

In [97]:
def min_and_max(cells, x_start):
    end = x_start.End(xlDown).Offset(1,2)
    rng = cells.Range(x_start, end)
    x, y = list(zip(*rng.Value))
    xmin = min(x)
    xmax = max(x)
    ymin = min(y)
    ymax = max(y)
    return xmin, xmax, ymin, ymax

def analyze_rtd_data(pth, title):
    xl = Excel()
    fp = os.path.abspath(pth)
    wb = xl.Workbooks.Open(fp)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    
    fudge_column(cells)
    amb, diff = make_amb_diff(cells)
    amb_start = amb.Offset(2,1)
    diff_start = diff.Offset(2,1)
    
    c = CreateChart(ws)
    x, y = chart_data_rng(cells, amb_start, diff_start)
    CreateDataSeries(c, x, y)
    FormatChart(c, None, title, "Ambient Temp(C)", "Err A-B", True, False)
    xmin, xmax, ymin, ymax = min_and_max(cells, amb_start)
    FormatAxesScale(c, int(xmin)-2, int(xmax)+2, 
                        round(ymin-.2, 1), round(ymax+.2, 1))
    c.Location(xlLocationAsNewSheet, "AmbRTD")
    c = wb.Charts("AmbRTD")

In [98]:
analyze_rtd_data("rtdtest.csv", "15L Ambient RTD Effect")